<a href="https://colab.research.google.com/github/elso6482/Visual-Question-Answering-Application-on-Health-and-Medical-Informatics/blob/main/IntUNITER_PathVQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/data_path.zip -d /content  

!unzip /content/drive/MyDrive/data/slakekg_embs.zip -d /content/data/
#!unzip /content/drive/MyDrive/data/slakekg_embs_restricted.zip
####!unzip /content/drive/MyDrive/data/knowledge_embeddings_restricted.zip /content/data/knowledge_embeddings
#%mv /content/knowledge_embeddings_restricted /content/data/knowledge_embeddings
#!unzip /content/drive/MyDrive/data/pvqa_slake_en_knowledge_BioBERT_rectified_v2.zip
#%mv /content/content/knowledge_embeddings_rectified_v2 /content/data/knowledge_embeddings

Streaming output truncated to the last 5000 lines.
  inflating: /content/data/knowledge_embeddings/29341.npy  
  inflating: /content/data/knowledge_embeddings/1517.npy  
  inflating: /content/data/knowledge_embeddings/2107.npy  
  inflating: /content/data/knowledge_embeddings/17234.npy  
  inflating: /content/data/knowledge_embeddings/11380.npy  
  inflating: /content/data/knowledge_embeddings/2244.npy  
  inflating: /content/data/knowledge_embeddings/17650.npy  
  inflating: /content/data/knowledge_embeddings/15462.npy  
  inflating: /content/data/knowledge_embeddings/30272.npy  
  inflating: /content/data/knowledge_embeddings/3852.npy  
  inflating: /content/data/knowledge_embeddings/15378.npy  
  inflating: /content/data/knowledge_embeddings/3143.npy  
  inflating: /content/data/knowledge_embeddings/32291.npy  
  inflating: /content/data/knowledge_embeddings/23288.npy  
  inflating: /content/data/knowledge_embeddings/6078.npy  
  inflating: /content/data/knowledge_embeddings/27467.n

In [ ]:
!unzip /content/drive/MyDrive/data/slakekg_embs_restricted.zip

%mv /content/knowledge_embeddings_restricted /content/data/knowledge_embeddings2

Streaming output truncated to the last 5000 lines.
  inflating: knowledge_embeddings_restricted/20261.npy  
  inflating: knowledge_embeddings_restricted/19296.npy  
  inflating: knowledge_embeddings_restricted/17205.npy  
  inflating: knowledge_embeddings_restricted/26619.npy  
  inflating: knowledge_embeddings_restricted/24798.npy  
  inflating: knowledge_embeddings_restricted/32018.npy  
  inflating: knowledge_embeddings_restricted/21834.npy  
  inflating: knowledge_embeddings_restricted/27215.npy  
  inflating: knowledge_embeddings_restricted/9449.npy  
  inflating: knowledge_embeddings_restricted/16096.npy  
  inflating: knowledge_embeddings_restricted/6150.npy  
  inflating: knowledge_embeddings_restricted/11773.npy  
  inflating: knowledge_embeddings_restricted/10034.npy  
  inflating: knowledge_embeddings_restricted/25150.npy  
  inflating: knowledge_embeddings_restricted/4804.npy  
  inflating: knowledge_embeddings_restricted/20416.npy  
  inflating: knowledge_embeddings_restri

In [ ]:
import torch
from torchvision import models
#model = models.resnet152(pretrained=True)
model = models.resnet50(pretrained=True)
feature_extractor = torch.nn.Sequential(*list(model.children())[:-2])
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
model_img = feature_extractor
model_img.eval()
img_crop = 224
i_model = 'resnet'

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
import torch
from torchvision import models
model = models.vgg16(pretrained=True)
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])

model_img = feature_extractor
model_img.eval()
img_crop = 224
i_model = 'vgg'

In [ ]:
import torch
from torchvision import models
model = models.densenet161(pretrained=True)
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])

model_img = feature_extractor
model_img.eval()
img_crop = 224
i_model = 'densenet'

In [ ]:
import torch
from torchvision import models
#feature_extractor = models.inception_v3(pretrained=True)
#feature_extractor = torch.nn.Sequential(*list(model.children())[:-5])

activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

feature_extractor = models.inception_v3(pretrained=True)
feature_extractor.Mixed_7b.register_forward_hook(get_activation('Mixed_7b'))
#out = feature_extractor(torch.randn(2, 3, 299, 299))


model_img = feature_extractor
model_img.eval()
img_crop = 299
i_model = 'inception'

In [ ]:
import torch
from PIL import Image
from torchvision import transforms
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(img_crop), #299 for inception 224 for others
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

if torch.cuda.is_available():    
    model_img.to(device)

def extract_visual_features(image):
    input = preprocess(image).unsqueeze(0).to(device)
    #print(input.size())
    if i_model =='resnet':
        output_img = torch.reshape(model_img(input).detach().transpose(1,3), [49,2048]).cpu().numpy() #Resnet (49, 2048)
    elif i_model =='vgg':
        output_img = torch.reshape(model_img(input).detach().transpose(1,3), [49,512]).cpu().numpy() #vgg (49,512)
    elif i_model =='densenet':
        output_img = torch.reshape(model_img(input).detach().transpose(1,3), [49,2208]).cpu().numpy() #dense (49,2208)
    elif i_model =='inception':
        output = model_img(input).detach().cpu().numpy() #Inception
        output_img = torch.reshape(activation['Mixed_7b'].detach(), [64,2048]).cpu().numpy() #Inception (64, 2048)
    else:
        raise ValueError
    return output_img

img_proposal = extract_visual_features(Image.open('/content/data/pic/0.jpg')).shape[0]
img_d = extract_visual_features(Image.open('/content/data/pic/0.jpg')).shape[1]
extract_visual_features(Image.open('/content/data/pic/0.jpg')).shape

(49, 2048)

In [ ]:
import numpy as np


for picture in os.listdir('/content/data/pic'):

    path = '/content/data/vis_features/'
    if not os.path.exists(path):
        os.mkdir(path)
    image_path = '/content/data/pic/' + picture
    save_path = path + picture.replace(".jpg","") + '.npy'
    print(picture)

    image = extract_visual_features(Image.open(image_path).convert('RGB'))
    np.save(save_path, image)

132.jpg
302.jpg
img_210.jpg
3075.jpg
676.jpg
Fig.314.jpg
2825.jpg
img_647.jpg
Fig.6.jpg
1607.jpg
2084.jpg
434.jpg
img_714.jpg
1499.jpg
3020.jpg
2191.jpg
Fig.82.jpg
911.jpg
img_565.jpg
1022.jpg
img_408.jpg
1953.jpg
1717.jpg
1231.jpg
235.jpg
1658.jpg
1318.jpg
827.jpg
2112.jpg
456.jpg
1450.jpg
321.jpg
1032.jpg
221.jpg
350.jpg
2445.jpg
243.jpg
2722.jpg
img_218.jpg
264.jpg
1436.jpg
976.jpg
img_691.jpg
247.jpg
440.jpg
img_605.jpg
1734.jpg
2201.jpg
2430.jpg
1870.jpg
img_831.jpg
img_22.jpg
589.jpg
1578.jpg
3275.jpg
3083.jpg
2533.jpg
1981.jpg
912.jpg
64.jpg
6.jpg
454.jpg
788.jpg
1132.jpg
Fig.808.jpg
1073.jpg
1485.jpg
2700.jpg
1106.jpg
Fig.69.jpg
1.jpg
2352.jpg
1797.jpg
98.jpg
1221.jpg
Fig.14.jpg
Fig.94.jpg
2511.jpg
img_361.jpg
img_271.jpg
3125.jpg
img_788.jpg
1918.jpg
img_314.jpg
752.jpg
1583.jpg
Fig.633.jpg
1172.jpg
3215.jpg
789.jpg
1136.jpg
Fig.310.jpg
1334.jpg
3088.jpg
Fig.629.jpg
3251.jpg
img_108.jpg
2624.jpg
1159.jpg
2252.jpg
948.jpg
Fig.705.jpg
3319.jpg
2853.jpg
464.jpg
3217.jpg
2224.jpg


In [ ]:
from sklearn.utils import shuffle
import pandas as pd
import pathlib
import numpy as np
answers_ids = {}
answers_ids['<pad>'] = 0
answers_ids['[UNKNOWN]'] = 1

#df = shuffle(pd.read_json('/content/data/jsons/QA.json'), random_state=1)
#df = shuffle(pd.read_json('/content/data/jsons/open_ended.json'), random_state=1)
df = shuffle(pd.read_json('/content/data/jsons/yes_no.json'), random_state=1)


DATASET_SIZE = len(df)
train_size = int(0.52 * DATASET_SIZE)
test_size = int(0.18 * DATASET_SIZE)


df_train_raw = df.iloc[:train_size]
df_test_raw = df.iloc[train_size:train_size+test_size]
df_validate_raw = df.iloc[train_size+test_size:]

iter = len(list(answers_ids.keys()))
# record answers and their ids
for answer in df_train_raw['Answers'].unique():
    answers_ids[answer] = iter
    iter+=1

In [ ]:
df_train_raw

,Question_Id,Images,Questions,Answers
24678,24702,1972,What does this image show?,skeletal muscle
20587,20608,1401,Is metastatic carcinoma x-ray present?,no
1320,1321,Fig.454,What shows that disease is limited to the mucosa?,the full-thickness histologic section
1513,1514,Fig.519,Are a few residual cardiac muscle cells presen...,no
17275,17293,1149,What removed?,opened base of skull with brain
...,...,...,...,...
11993,12005,651,Where is this part in the figure?,endocrine system
21484,21508,1496,Is optic nerve present?,no
17327,17345,1153,What is present ?,metastatic carcinoma
2537,2539,img_41,Where is the affected area?,on right


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.3 MB 4.0 MB/s 
     |████████████████████████████████| 596 kB 93.7 MB/s 
     |████████████████████████████████| 61 kB 604 kB/s 
     |████████████████████████████████| 895 kB 95.6 MB/s 
     |████████████████████████████████| 3.3 MB 84.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# prepare all datasets
from transformers import BertTokenizer, BertConfig
import torch
from torchsummary import summary
import torch.nn as nn

max_length = 20

tokenizer_bert = BertTokenizer.from_pretrained('bert-base-uncased')
vocab_size = len(list(tokenizer_bert.vocab.keys()))

def encode_to_id(string_sequence):
    encoded_input = tokenizer_bert(string_sequence, padding='max_length', truncation=True, max_length=max_length)
    #tokenised = tokenizer_bert.tokenize(string_sequence)
    #print(encoded_input)
    indexed_tokens = encoded_input['input_ids']
    segments_ids = encoded_input['token_type_ids']
    attention_mask = encoded_input["attention_mask"]
    position_ids = list(range(max_length))
    return [indexed_tokens, segments_ids, attention_mask, position_ids]


def encode_answer(dictionary_answers, answer):
    if answer in dictionary_answers:
        return dictionary_answers[answer]
    else:
        return 0


def question_encoding(qid):
    return str(qid) + '.npy'

def knowledge_encoding(split, qid):
    return str(qid) + '.npy'






In [ ]:
a = encode_to_id("I am a boy")
a

[[101, 1045, 2572, 1037, 2879, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]]

In [ ]:
df_train, df_validate, df_test = pd.DataFrame(), pd.DataFrame(), pd.DataFrame() 

df_train['img_name'] = df_train_raw['Images'].apply(lambda x: x + '.npy')
df_validate['img_name'] = df_validate_raw['Images'].apply(lambda x: x + '.npy')
df_test['img_name'] = df_test_raw['Images'].apply(lambda x: x + '.npy')

df_train['question_ids'] = df_train_raw['Questions'].apply(lambda x: encode_to_id(x)[0])
df_validate['question_ids'] = df_validate_raw['Questions'].apply(lambda x: encode_to_id(x)[0])
df_test['question_ids'] = df_test_raw['Questions'].apply(lambda x: encode_to_id(x)[0])

df_train['question_type'] = df_train_raw['Questions'].apply(lambda x: encode_to_id(x)[1])
df_validate['question_type'] = df_validate_raw['Questions'].apply(lambda x: encode_to_id(x)[1])
df_test['question_type'] = df_test_raw['Questions'].apply(lambda x: encode_to_id(x)[1])

df_train['question_position_ids'] = df_train_raw['Questions'].apply(lambda x: encode_to_id(x)[3])
df_validate['question_position_ids'] = df_validate_raw['Questions'].apply(lambda x: encode_to_id(x)[3])
df_test['question_position_ids'] = df_test_raw['Questions'].apply(lambda x: encode_to_id(x)[3])

df_train['question_encoded'] = df_train_raw['Question_Id'].apply(lambda x: question_encoding(x))
df_validate['question_encoded'] = df_validate_raw['Question_Id'].apply(lambda x: question_encoding(x))
df_test['question_encoded'] = df_test_raw['Question_Id'].apply(lambda x: question_encoding(x))

df_train['knowledge'] = df_train_raw['Question_Id'].apply(lambda x: knowledge_encoding('train', x))
df_validate['knowledge'] = df_validate_raw['Question_Id'].apply(lambda x: knowledge_encoding('validate', x))
df_test['knowledge'] = df_test_raw['Question_Id'].apply(lambda x: knowledge_encoding('test', x))



df_train['answer_encoded'] = df_train_raw['Answers'].apply(lambda x: encode_answer(answers_ids, x))
df_validate['answer_encoded'] = df_validate_raw['Answers'].apply(lambda x: encode_answer(answers_ids, x))
df_test['answer_encoded'] = df_test_raw['Answers'].apply(lambda x: encode_answer(answers_ids, x))

vocab_size =len(list(tokenizer_bert.vocab.keys()))

In [ ]:
print(len(list(tokenizer_bert.vocab.keys())))

30522


In [ ]:
df_train.head()

,img_name,question_ids,question_type,question_position_ids,question_encoded,knowledge,answer_encoded
24678,1972.npy,"[101, 2054, 2515, 2023, 3746, 2265, 1029, 102,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",24702.npy,24702.npy,1
20587,1401.npy,"[101, 2003, 18804, 16677, 2482, 21081, 2863, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",20608.npy,20608.npy,2
1320,Fig.454.npy,"[101, 2054, 3065, 2008, 4295, 2003, 3132, 2000...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",1321.npy,1321.npy,3
1513,Fig.519.npy,"[101, 2024, 1037, 2261, 21961, 15050, 6740, 44...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",1514.npy,1514.npy,2
17275,1149.npy,"[101, 2054, 3718, 1029, 102, 0, 0, 0, 0, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",17293.npy,17293.npy,4


In [ ]:
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchvision import transforms, utils
import os


class PathVQA_Dataset(Dataset):
    def __init__(self, df, root_dir, kn_dir, q_dir):
        """
        Args:
            df (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            kn_dir (string): Directory with knowledge embeddings (like BERT etc)
            q_dir (string): Directory with question embeddings (like BioELMo etc)
        """
        self.questions_file = df
        self.root_dir = root_dir
        self.kn_dir = kn_dir
        self.q_dir = q_dir


    def __len__(self):
        return len(self.questions_file)


    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.questions_file.iloc[idx, 0])
        image = np.load(img_name)

        question_type = np.array(self.questions_file.iloc[idx, 2])
        question_pos = np.array(self.questions_file.iloc[idx, 3])

        question = self.questions_file.iloc[idx, 1]

        question_encoded_name = os.path.join(self.q_dir,
                                             self.questions_file.iloc[idx, 4])
        question_encoded = np.load(question_encoded_name)
        question_encoded = np.pad(question_encoded, ((0,38-question_encoded.shape[0]), 
                                                     (0,0)), 'constant')

        knowledge_encoded_name = os.path.join(self.kn_dir,
                                              self.questions_file.iloc[idx, 5])
        knowledge = np.load(knowledge_encoded_name)
        if knowledge.shape[0]<192:
            knowledge = np.pad(knowledge, ((0,192-knowledge.shape[0]), 
                                                        (0,0)), 'constant')
        else:
            knowledge = knowledge[:192]

        answer = self.questions_file.iloc[idx, 6]

        pos = torch.tensor([0.,0.,1.,1.]).repeat(img_proposal,1)

        sample = {'image': image, 'position':pos, 'question_pos':question_pos, 'question_type':question_type,
                  'question': question, 'question_encoded':question_encoded,
                  'knowledge': knowledge, 'answer': answer}

        return sample




class SLAKE_Dataset(Dataset):
    def __init__(self, df, root_dir, kn_dir, q_dir):
        """
        Args:
            df (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            kn_dir (string): Directory with knowledge embeddings (like BERT etc)
            q_dir (string): Directory with question embeddings (like BioELMo etc)
        """
        self.questions_file = df
        self.root_dir = root_dir
        self.kn_dir = kn_dir
        self.q_dir = q_dir


    def __len__(self):
        return len(self.questions_file)


    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.questions_file.iloc[idx, 0])
        image = np.load(img_name)

        question_type = np.array(self.questions_file.iloc[idx, 2])
        question_pos = np.array(self.questions_file.iloc[idx, 3])

        question = self.questions_file.iloc[idx, 1]

        question_encoded_name = os.path.join(self.q_dir,
                                             self.questions_file.iloc[idx, 4])
        question_encoded = np.load(question_encoded_name)
        question_encoded = np.pad(question_encoded, ((0,38-question_encoded.shape[0]), 
                                                     (0,0)), 'constant')

        knowledge_encoded_name = os.path.join(self.kn_dir,
                                              self.questions_file.iloc[idx, 5])
        knowledge = np.load(knowledge_encoded_name)
        if knowledge.shape[0]<192:
            knowledge = np.pad(knowledge, ((0,192-knowledge.shape[0]), 
                                                        (0,0)), 'constant')
        else:
            knowledge = knowledge[:192]

        answer = self.questions_file.iloc[idx, 6]

        pos = torch.tensor([0.,0.,1.,1.]).repeat(img_proposal,1)

        sample = {'image': image, 'position':pos, 'question_pos':question_pos, 'question_type':question_type,
                  'question': question, 'question_encoded':question_encoded,
                  'knowledge': knowledge, 'answer': answer}

        return sample

In [ ]:
BATCH_SIZE = 16

train_dataloader = DataLoader(PathVQA_Dataset(df_train, '/content/data/vis_features', 
                                            '/content/data/knowledge_embeddings', 
                                            '/content/data/ques_embeddings/bioelmo'), 
                              batch_size=BATCH_SIZE, shuffle = True, pin_memory=True)
test_dataloader = DataLoader(PathVQA_Dataset(df_test, '/content/data/vis_features', 
                                            '/content/data/knowledge_embeddings', 
                                            '/content/data/ques_embeddings/bioelmo'),
                             batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)
val_dataloader = DataLoader(PathVQA_Dataset(df_validate, '/content/data/vis_features', 
                                            '/content/data/knowledge_embeddings', 
                                            '/content/data/ques_embeddings/bioelmo'),
                            batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

In [ ]:
knowledge1 = np.load('/content/data/knowledge_embeddings/6.npy')
knowledge2 = np.load('/content/data/knowledge_embeddings2/6.npy')
knowledge2 = np.pad(knowledge2, ((0,192-knowledge2.shape[0]), 
                                              (0,0)), 'constant')
print(knowledge1[:192].shape)
print(knowledge2.shape)
a =[1,2,3,4]
a[:2]

(192, 1024)
(192, 1024)


[1, 2]

In [ ]:
idx, batch = next(enumerate(train_dataloader))
print(torch.stack(batch['question']).transpose(0,1).size())
print(batch['image'].size())
print(batch['position'].size())
print(batch['question_encoded'].size())
print(batch['question_pos'].size())
print(batch['question_type'].size())
print(batch['knowledge'].size())
print(batch['answer'].size())

torch.Size([16, 20])
torch.Size([16, 64, 2048])
torch.Size([16, 64, 4])
torch.Size([16, 38, 1024])
torch.Size([16, 20])
torch.Size([16, 20])
torch.Size([16, 192, 1024])
torch.Size([16])


In [ ]:
def gelu(x):

    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

In [ ]:
HIDDEN_SIZE = 192
INTERMEDIATE_SIZE = 192
NUMBER_OF_LAYER = 3

from torch import nn
class UniterConfig(object):

    def __init__(self, vocab_size, hidden_size=HIDDEN_SIZE, num_hidden_layers=NUMBER_OF_LAYER, num_attention_heads=12, intermediate_size=INTERMEDIATE_SIZE, hidden_act="gelu",
                 hidden_dropout_prob=0.1, attention_probs_dropout_prob=0.1,
                 max_position_embeddings=64, type_vocab_size=2, initializer_range=0.02):
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.num_hidden_layers = num_hidden_layers
        self.num_attention_heads = num_attention_heads
        self.hidden_act = hidden_act
        self.intermediate_size = intermediate_size
        self.hidden_dropout_prob = hidden_dropout_prob
        self.attention_probs_dropout_prob = attention_probs_dropout_prob
        self.max_position_embeddings = max_position_embeddings
        self.type_vocab_size = type_vocab_size
        self.initializer_range = initializer_range

config = UniterConfig(vocab_size)

In [ ]:
class UniterTextEmbeddings(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.word_embeddings = nn.Embedding(config.vocab_size,
                                            config.hidden_size, padding_idx=0)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings,
                                                config.hidden_size)
        self.token_type_embeddings = nn.Embedding(config.type_vocab_size,
                                                  config.hidden_size)

        # self.LayerNorm is not snake-cased to stick with TensorFlow model
        # variable name and be able to load any TensorFlow checkpoint file
        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=1e-12)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, input_ids, position_ids, token_type_ids=None):
        if token_type_ids==None:
            token_type_ids = torch.zeros_like(input_ids)

        words_embeddings = self.word_embeddings(input_ids)
        position_embeddings = self.position_embeddings(position_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)

        #print(input_ids.size())
        #print(token_type_ids.size())
        #print(words_embeddings.size())
        #print(position_embeddings.size())
        #print(token_type_embeddings.size())  
        #print("-----------")      

        embeddings = (words_embeddings
                      + position_embeddings
                      + token_type_embeddings)
      


        #print(embeddings)
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings

In [ ]:
class UniterImageEmbeddings(nn.Module):
    def __init__(self, config, img_dim):
        super().__init__()
        self.img_linear = nn.Linear(img_dim, config.hidden_size)
        self.img_layer_norm = nn.LayerNorm(config.hidden_size, eps=1e-12)
        self.pos_layer_norm = nn.LayerNorm(config.hidden_size, eps=1e-12)
        self.pos_linear = nn.Linear(4, config.hidden_size)
        self.mask_embedding = nn.Embedding(2, img_dim, padding_idx=0)

        # tf naming convention for layer norm
        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=1e-12)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, img_feat, img_pos_feat, type_embeddings, img_masks=None):
        if img_masks is not None:
            self.mask_embedding.weight.data[0, :].fill_(0)
            mask = self.mask_embedding(img_masks.long())
            img_feat = img_feat + mask

        transformed_im = self.img_layer_norm(self.img_linear(img_feat))
        transformed_pos = self.pos_layer_norm(self.pos_linear(img_pos_feat))
        embeddings = transformed_im + transformed_pos + type_embeddings
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)

        return embeddings

In [ ]:
class BertPooler(nn.Module):
    def __init__(self, config):
        super(BertPooler, self).__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()

    def forward(self, hidden_states):
        # We "pool" the model by simply taking the hidden state corresponding
        # to the first token.
        first_token_tensor = hidden_states[:, 0]
        pooled_output = self.dense(first_token_tensor)
        pooled_output = self.activation(pooled_output)
        return pooled_output

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model = 4, dropout = 0.1, max_len = 70):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [ ]:

import math
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

class TransformerModel(nn.Module):

    def __init__(self, d_model, nhead, d_hid,
                 nlayers, dropout, max_len):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout, max_len)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        #self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        #self.decoder = nn.Linear(d_model, ntoken)

        #self.init_weights()

    #def init_weights(self) -> None:
    #    initrange = 0.1
    #    self.encoder.weight.data.uniform_(-initrange, initrange)
    #    self.decoder.bias.data.zero_()
    #    self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = src.permute(1,0,2)* math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        #print("src_size:", src.size())
        output = self.transformer_encoder(src)
        #output = self.decoder(output)
        return output


In [ ]:
class UniterModel(nn.Module):
    """ Modification for Joint Vision-Language Encoding
    """
    def __init__(self, config, img_dim):
        super(UniterModel, self).__init__()
        self.embeddings = UniterTextEmbeddings(config)
        self.img_embeddings = UniterImageEmbeddings(config, img_dim)
        self.transformer = TransformerModel(d_model=192, nhead=config.num_attention_heads, 
                      d_hid=config.hidden_size, nlayers=config.num_hidden_layers, 
                      dropout=config.hidden_dropout_prob, max_len = 1 + img_proposal + max_length)
        #self.encoder = UniterEncoder(config)
        #self.pooler = BertPooler(config)
        #self.apply(self.init_weights)
        self.dense_pool = nn.Linear(config.hidden_size, config.hidden_size)
        self.dense_img = nn.Linear(config.hidden_size, config.hidden_size)
        self.dense_visual = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()
        self.activation_pool = nn.GELU()


    def _compute_txt_embeddings(self, input_ids, position_ids,
                                txt_type_ids=None):
        output = self.embeddings(input_ids, position_ids, txt_type_ids)
        return output

    def _compute_img_embeddings(self, img_feat, img_pos_feat, img_masks=None,
                                img_type_ids=None):
        if img_type_ids is None:
            img_type_ids = torch.ones_like(img_feat[:, :, 0].long())
        img_type_embeddings = self.embeddings.token_type_embeddings(
            img_type_ids)
        output = self.img_embeddings(img_feat, img_pos_feat,
                                     img_type_embeddings, img_masks)
        return output

    def _compute_img_txt_embeddings(self, input_ids, position_ids,
                                    img_feat, img_pos_feat,
                                    img_masks=None,
                                    txt_type_ids=None, img_type_ids=None):
        txt_emb = self._compute_txt_embeddings(
            input_ids, position_ids, txt_type_ids)
        img_emb = self._compute_img_embeddings(
            img_feat, img_pos_feat, img_masks, img_type_ids)
        # align back to most compact input
        pool_feature = torch.zeros(1, 1, config.hidden_size).repeat(txt_emb.size()[0],1,1).to(device)
        #print(pool_feature.size())
        #print(img_emb.size())
        #print(txt_emb.size())
        transformer_input = torch.cat([pool_feature, img_emb, txt_emb], dim=1)
        #print(transformer_input.size())
        output = self.transformer(transformer_input).permute(1,0,2)
        #print(output.size())
        return output

    def forward(self, input_ids, position_ids,
                img_feat, img_pos_feat, gather_index=None, img_masks=None,
                output_all_encoded_layers=True,
                txt_type_ids=None, img_type_ids=None):
        # compute self-attention mask
        #extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
        #extended_attention_mask = extended_attention_mask.to(
        #    dtype=next(self.parameters()).dtype)  # fp16 compatibility
        #extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0

        # embedding layer
        if input_ids is None:
            # image only
            embedding_output = self._compute_img_embeddings(
                img_feat, img_pos_feat, img_masks, img_type_ids)
        elif img_feat is None:
            # text only
            embedding_output = self._compute_txt_embeddings(
                input_ids, position_ids, txt_type_ids)
        else:
            embedding_output = self._compute_img_txt_embeddings(
                input_ids, position_ids,
                img_feat, img_pos_feat,
                img_masks, txt_type_ids, img_type_ids)



        pool_embedding = embedding_output[:, 0]
        pooled_output = self.dense_pool(pool_embedding)
        pooled_output = self.activation_pool(pooled_output)

        img_embedding = embedding_output[:, 1:img_feat.size()[1]+1]

        img_output = self.dense_img(img_embedding)
        img_output = self.activation(img_output)

        textual_embedding = embedding_output[:, img_feat.size()[1]+1:]
        textual_output = self.dense_visual(textual_embedding)
        textual_output = self.activation(textual_output)

        #if not output_all_encoded_layers:
        #    encoded_layers = encoded_layers[-1]

        return pooled_output, img_output, textual_output, embedding_output

In [ ]:
#No image
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())
        self.wk = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())
        self.wv = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())

        self.dense = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())

    def positional_encoding(self, position, d_model):
        def get_angles(pos, i, d_model):
            angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
            return pos * angle_rates
        angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                                np.arange(d_model)[np.newaxis, :],
                                d_model)

        # apply sin to even indices in the array; 2i
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

        # apply cos to odd indices in the array; 2i+1
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

        pos_encoding = angle_rads[np.newaxis, ...]
        return torch.Tensor(pos_encoding)
    
    def scaled_dot_product_attention(self, q, k, v):
        matmul_qk = torch.matmul(q, torch.transpose(k, 2, 3))
        dk = matmul_qk.shape[-1]
        dk = torch.sqrt(torch.tensor(dk).to(device))+1e-6
        scaled_attention_logits = matmul_qk / dk
        attention_weights = F.softmax(scaled_attention_logits, dim=-1)
        output = torch.matmul(attention_weights, v)  

        return output, attention_weights 

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = torch.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return x.permute([0, 2, 1, 3])

    def forward(self, v, k, q):  # Note: original code call(self, v, k, q, mask):
        batch_size = q.shape[0]
        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)
        
        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
        
        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = self.scaled_dot_product_attention(
            q, k, v)

        scaled_attention = torch.permute(scaled_attention, (0, 2, 1, 3))  # (batch_size, seq_len_q, num_heads, depth)
        concat_attention = torch.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)
        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights


class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, dff, batch_size, 
                 maximum_position_encoding, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.layers_list = [nn.Linear(d_model, dff),nn.ReLU(),nn.Linear(dff, d_model)]
        self.ffn = nn.Sequential(*self.layers_list)
        self.maximum_position_encoding = int(maximum_position_encoding)

        self.layernorm1 = nn.LayerNorm(normalized_shape=(
                                                         maximum_position_encoding,d_model), eps=1e-6)
        self.layernorm2 = nn.LayerNorm(normalized_shape=(
                                                         maximum_position_encoding,d_model), eps=1e-6)

        self.dropout1 = nn.Dropout(rate)
        self.dropout2 = nn.Dropout(rate)

    def forward(self, question, img):
        attn_output, attn_weights_block = self.mha(img, question, question) 
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(question + attn_output)  

        
        ffn_output = self.ffn(out1)  
        ffn_output = self.dropout2(ffn_output)
        out2 = self.layernorm2(out1 + ffn_output)  

        return out2, attn_weights_block



class Encoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff,
                 maximum_position_encoding, batch_size, rate=0.1):
        super().__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.batch_size = batch_size

        self.question_dense = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())
        
        #self.pos_encoding = torch.Tensor(positional_encoding(maximum_position_encoding,
        #                                        self.d_model)).to(device)

        self.enc_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, 
                                                      dff, self.batch_size, maximum_position_encoding,
                                                      rate)
                           for _ in range(int(num_layers))])
        self.dropout = nn.Dropout(rate)
    
    def forward(self, question, img):
        seq_len = question.shape[1]
        x = self.question_dense(question)
        #x *= torch.sqrt(torch.tensor(self.d_model).to(device))
        x = torch.mul(x, torch.sqrt(torch.tensor(self.d_model).to(device)))

        #x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x)
        

        attention_weights = {}

        for i in range(self.num_layers):
            x, block = self.enc_layers[i](x, img)
            attention_weights['encoder_layer{}_block'.format(i + 1)] = block
        return x, attention_weights  # (batch_size, input_seq_len, d_model)


class Knowledge_Question_Encoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff, pe_input, rate, batch_size,
                 multiplier=10):
        super().__init__()
        self.batch_size = batch_size
        self.maximum_position_encoding = pe_input
        self.multiplier = multiplier
        self.bilstm_dim = int(d_model/2)
        self.bilstm1 = nn.LSTM(1024, self.bilstm_dim, batch_first=True, bidirectional=True)
        self.bilstm2 = nn.LSTM(1024, self.bilstm_dim, batch_first=True, bidirectional=True)

        self.fc1 = nn.Linear(d_model, self.maximum_position_encoding * multiplier)
        self.kn_dense = nn.Sequential(nn.Linear(192 * multiplier, d_model), nn.ReLU())

        self.question_encoder = Encoder(
            num_layers=num_layers,
            d_model=d_model,
            num_heads=num_heads,
            dff=dff,
            maximum_position_encoding=self.maximum_position_encoding,
            batch_size=self.batch_size)
        
    def forward(self, question, kn, batch_size):
        question, _ = self.bilstm1(question) #(batch, 38, 336)
        kn, _ = self.bilstm2(kn) #(batch, 192, 336)
        kn = self.fc1(kn)
        kn = torch.reshape(kn, (kn.shape[0], self.maximum_position_encoding, 
                                192 * self.multiplier))
        kn = self.kn_dense(kn)

        context, _ = self.question_encoder(question, kn)
        return context



class Uniter_classifier(nn.Module):
    def __init__(self, model, ans_size):
        super(Uniter_classifier, self).__init__()
        self.batch_size = BATCH_SIZE
        self.knowledge_encoder = Knowledge_Question_Encoder(1, HIDDEN_SIZE, 12, 
                                                            INTERMEDIATE_SIZE, 
                                                            38, 0.1, self.batch_size)
        self.pre_trained = model

        self.classifier1 = nn.Sequential(nn.Linear(38+img_proposal, 1), nn.ReLU())
        self.classifier2 = nn.Linear(HIDDEN_SIZE, ans_size)


    def forward(self, inputs_ids, question_pos, visual_feats, visual_pos, question_encoded, 
                knowledge):
 
        classification_output, vision_output, _, _ = self.pre_trained(inputs_ids, question_pos, visual_feats*0, visual_pos)

        #classification_output = classification_output.unsqueeze(1)
        #print(classification_output.size())
        #print(vision_output.size())
        #print(knowledge.size())
        knowledge_encoded = self.knowledge_encoder(question_encoded, knowledge, 
                                                   self.batch_size)


        concatenated_output = torch.cat((vision_output, knowledge_encoded), 1)
        concatenated_output = torch.permute(concatenated_output, (0, 2, 1))
        #print(concatenated_output.size())
        output = self.classifier1(concatenated_output).squeeze(-1)
        prediction = self.classifier2(output + classification_output)
        
        return prediction

UniterM = UniterModel(config, img_d)
model_Uniter = Uniter_classifier(UniterM, len(list(answers_ids.keys()))).to(device)

In [ ]:
#Only question
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())
        self.wk = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())
        self.wv = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())

        self.dense = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())

    def positional_encoding(self, position, d_model):
        def get_angles(pos, i, d_model):
            angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
            return pos * angle_rates
        angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                                np.arange(d_model)[np.newaxis, :],
                                d_model)

        # apply sin to even indices in the array; 2i
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

        # apply cos to odd indices in the array; 2i+1
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

        pos_encoding = angle_rads[np.newaxis, ...]
        return torch.Tensor(pos_encoding)
    
    def scaled_dot_product_attention(self, q, k, v):
        matmul_qk = torch.matmul(q, torch.transpose(k, 2, 3))
        dk = matmul_qk.shape[-1]
        dk = torch.sqrt(torch.tensor(dk).to(device))+1e-6
        scaled_attention_logits = matmul_qk / dk
        attention_weights = F.softmax(scaled_attention_logits, dim=-1)
        output = torch.matmul(attention_weights, v)  

        return output, attention_weights 

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = torch.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return x.permute([0, 2, 1, 3])

    def forward(self, v, k, q):  # Note: original code call(self, v, k, q, mask):
        batch_size = q.shape[0]
        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)
        
        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
        
        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = self.scaled_dot_product_attention(
            q, k, v)

        scaled_attention = torch.permute(scaled_attention, (0, 2, 1, 3))  # (batch_size, seq_len_q, num_heads, depth)
        concat_attention = torch.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)
        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights


class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, dff, batch_size, 
                 maximum_position_encoding, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.layers_list = [nn.Linear(d_model, dff),nn.ReLU(),nn.Linear(dff, d_model)]
        self.ffn = nn.Sequential(*self.layers_list)
        self.maximum_position_encoding = int(maximum_position_encoding)

        self.layernorm1 = nn.LayerNorm(normalized_shape=(
                                                         maximum_position_encoding,d_model), eps=1e-6)
        self.layernorm2 = nn.LayerNorm(normalized_shape=(
                                                         maximum_position_encoding,d_model), eps=1e-6)

        self.dropout1 = nn.Dropout(rate)
        self.dropout2 = nn.Dropout(rate)

    def forward(self, question, img):
        attn_output, attn_weights_block = self.mha(img, question, question) 
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(question + attn_output)  

        
        ffn_output = self.ffn(out1)  
        ffn_output = self.dropout2(ffn_output)
        out2 = self.layernorm2(out1 + ffn_output)  

        return out2, attn_weights_block



class Encoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff,
                 maximum_position_encoding, batch_size, rate=0.1):
        super().__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.batch_size = batch_size

        self.question_dense = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())
        
        #self.pos_encoding = torch.Tensor(positional_encoding(maximum_position_encoding,
        #                                        self.d_model)).to(device)

        self.enc_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, 
                                                      dff, self.batch_size, maximum_position_encoding,
                                                      rate)
                           for _ in range(int(num_layers))])
        self.dropout = nn.Dropout(rate)
    
    def forward(self, question, img):
        seq_len = question.shape[1]
        x = self.question_dense(question)
        #x *= torch.sqrt(torch.tensor(self.d_model).to(device))
        x = torch.mul(x, torch.sqrt(torch.tensor(self.d_model).to(device)))

        #x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x)
        

        attention_weights = {}

        for i in range(self.num_layers):
            x, block = self.enc_layers[i](x, img)
            attention_weights['encoder_layer{}_block'.format(i + 1)] = block
        return x, attention_weights  # (batch_size, input_seq_len, d_model)


class Knowledge_Question_Encoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff, pe_input, rate, batch_size,
                 multiplier=10):
        super().__init__()
        self.batch_size = batch_size
        self.maximum_position_encoding = pe_input
        self.multiplier = multiplier
        self.bilstm_dim = int(d_model/2)
        self.bilstm1 = nn.LSTM(1024, self.bilstm_dim, batch_first=True, bidirectional=True)
        self.bilstm2 = nn.LSTM(1024, self.bilstm_dim, batch_first=True, bidirectional=True)

        self.fc1 = nn.Linear(d_model, self.maximum_position_encoding * multiplier)
        self.kn_dense = nn.Sequential(nn.Linear(192 * multiplier, d_model), nn.ReLU())

        self.question_encoder = Encoder(
            num_layers=num_layers,
            d_model=d_model,
            num_heads=num_heads,
            dff=dff,
            maximum_position_encoding=self.maximum_position_encoding,
            batch_size=self.batch_size)
        
    def forward(self, question, kn, batch_size):
        question, _ = self.bilstm1(question) #(batch, 38, 336)
        kn, _ = self.bilstm2(kn) #(batch, 192, 336)
        kn = self.fc1(kn)
        kn = torch.reshape(kn, (kn.shape[0], self.maximum_position_encoding, 
                                192 * self.multiplier))
        kn = self.kn_dense(kn)

        context, _ = self.question_encoder(question, kn)
        return context



class Uniter_classifier(nn.Module):
    def __init__(self, model, ans_size):
        super(Uniter_classifier, self).__init__()
        self.batch_size = BATCH_SIZE
        self.knowledge_encoder = Knowledge_Question_Encoder(1, HIDDEN_SIZE, 12, 
                                                            INTERMEDIATE_SIZE, 
                                                            38, 0.1, self.batch_size)
        self.pre_trained = model

        self.classifier1 = nn.Sequential(nn.Linear(38+img_proposal, 1), nn.ReLU())
        self.classifier2 = nn.Linear(HIDDEN_SIZE, ans_size)


    def forward(self, inputs_ids, question_pos, visual_feats, visual_pos, question_encoded, 
                knowledge):
 
        classification_output, vision_output, _, _ = self.pre_trained(inputs_ids, question_pos, visual_feats*0, visual_pos)

        #classification_output = classification_output.unsqueeze(1)
        #print(classification_output.size())
        #print(vision_output.size())
        #print(knowledge.size())
        knowledge_encoded = self.knowledge_encoder(question_encoded, knowledge, 
                                                   self.batch_size)


        concatenated_output = torch.cat((vision_output, knowledge_encoded*0), 1)
        concatenated_output = torch.permute(concatenated_output, (0, 2, 1))
        #print(concatenated_output.size())
        output = self.classifier1(concatenated_output).squeeze(-1)
        prediction = self.classifier2(output + classification_output)
        
        return prediction

UniterM = UniterModel(config, img_d)
model_Uniter = Uniter_classifier(UniterM, len(list(answers_ids.keys()))).to(device)

In [ ]:
#No question
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())
        self.wk = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())
        self.wv = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())

        self.dense = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())

    def positional_encoding(self, position, d_model):
        def get_angles(pos, i, d_model):
            angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
            return pos * angle_rates
        angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                                np.arange(d_model)[np.newaxis, :],
                                d_model)

        # apply sin to even indices in the array; 2i
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

        # apply cos to odd indices in the array; 2i+1
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

        pos_encoding = angle_rads[np.newaxis, ...]
        return torch.Tensor(pos_encoding)
    
    def scaled_dot_product_attention(self, q, k, v):
        matmul_qk = torch.matmul(q, torch.transpose(k, 2, 3))
        dk = matmul_qk.shape[-1]
        dk = torch.sqrt(torch.tensor(dk).to(device))+1e-6
        scaled_attention_logits = matmul_qk / dk
        attention_weights = F.softmax(scaled_attention_logits, dim=-1)
        output = torch.matmul(attention_weights, v)  

        return output, attention_weights 

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = torch.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return x.permute([0, 2, 1, 3])

    def forward(self, v, k, q):  # Note: original code call(self, v, k, q, mask):
        batch_size = q.shape[0]
        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)
        
        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
        
        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = self.scaled_dot_product_attention(
            q, k, v)

        scaled_attention = torch.permute(scaled_attention, (0, 2, 1, 3))  # (batch_size, seq_len_q, num_heads, depth)
        concat_attention = torch.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)
        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights


class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, dff, batch_size, 
                 maximum_position_encoding, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.layers_list = [nn.Linear(d_model, dff),nn.ReLU(),nn.Linear(dff, d_model)]
        self.ffn = nn.Sequential(*self.layers_list)
        self.maximum_position_encoding = int(maximum_position_encoding)

        self.layernorm1 = nn.LayerNorm(normalized_shape=(
                                                         maximum_position_encoding,d_model), eps=1e-6)
        self.layernorm2 = nn.LayerNorm(normalized_shape=(
                                                         maximum_position_encoding,d_model), eps=1e-6)

        self.dropout1 = nn.Dropout(rate)
        self.dropout2 = nn.Dropout(rate)

    def forward(self, question, img):
        attn_output, attn_weights_block = self.mha(img, question, question) 
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(question + attn_output)  

        
        ffn_output = self.ffn(out1)  
        ffn_output = self.dropout2(ffn_output)
        out2 = self.layernorm2(out1 + ffn_output)  

        return out2, attn_weights_block



class Encoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff,
                 maximum_position_encoding, batch_size, rate=0.1):
        super().__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.batch_size = batch_size

        self.question_dense = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())
        
        #self.pos_encoding = torch.Tensor(positional_encoding(maximum_position_encoding,
        #                                        self.d_model)).to(device)

        self.enc_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, 
                                                      dff, self.batch_size, maximum_position_encoding,
                                                      rate)
                           for _ in range(int(num_layers))])
        self.dropout = nn.Dropout(rate)
    
    def forward(self, question, img):
        seq_len = question.shape[1]
        x = self.question_dense(question)
        #x *= torch.sqrt(torch.tensor(self.d_model).to(device))
        x = torch.mul(x, torch.sqrt(torch.tensor(self.d_model).to(device)))

        #x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x)
        

        attention_weights = {}

        for i in range(self.num_layers):
            x, block = self.enc_layers[i](x, img)
            attention_weights['encoder_layer{}_block'.format(i + 1)] = block
        return x, attention_weights  # (batch_size, input_seq_len, d_model)


class Knowledge_Question_Encoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff, pe_input, rate, batch_size,
                 multiplier=10):
        super().__init__()
        self.batch_size = batch_size
        self.maximum_position_encoding = pe_input
        self.multiplier = multiplier
        self.bilstm_dim = int(d_model/2)
        self.bilstm1 = nn.LSTM(1024, self.bilstm_dim, batch_first=True, bidirectional=True)
        self.bilstm2 = nn.LSTM(1024, self.bilstm_dim, batch_first=True, bidirectional=True)

        self.fc1 = nn.Linear(d_model, self.maximum_position_encoding * multiplier)
        self.kn_dense = nn.Sequential(nn.Linear(192 * multiplier, d_model), nn.ReLU())

        self.question_encoder = Encoder(
            num_layers=num_layers,
            d_model=d_model,
            num_heads=num_heads,
            dff=dff,
            maximum_position_encoding=self.maximum_position_encoding,
            batch_size=self.batch_size)
        
    def forward(self, question, kn, batch_size):
        question, _ = self.bilstm1(question) #(batch, 38, 336)
        kn, _ = self.bilstm2(kn) #(batch, 192, 336)
        kn = self.fc1(kn)
        kn = torch.reshape(kn, (kn.shape[0], self.maximum_position_encoding, 
                                192 * self.multiplier))
        kn = self.kn_dense(kn)

        context, _ = self.question_encoder(question, kn)
        return context



class Uniter_classifier(nn.Module):
    def __init__(self, model, ans_size):
        super(Uniter_classifier, self).__init__()
        self.batch_size = BATCH_SIZE
        self.knowledge_encoder = Knowledge_Question_Encoder(1, HIDDEN_SIZE, 12, 
                                                            INTERMEDIATE_SIZE, 
                                                            38, 0.1, self.batch_size)
        self.pre_trained = model

        self.classifier1 = nn.Sequential(nn.Linear(38+img_proposal, 1), nn.ReLU())
        self.classifier2 = nn.Linear(HIDDEN_SIZE, ans_size)


    def forward(self, inputs_ids, question_pos, visual_feats, visual_pos, question_encoded, 
                knowledge):
 
        classification_output, vision_output, _, _ = self.pre_trained(inputs_ids*0, question_pos, visual_feats, visual_pos)

        #classification_output = classification_output.unsqueeze(1)
        #print(classification_output.size())
        #print(vision_output.size())
        #print(knowledge.size())
        knowledge_encoded = self.knowledge_encoder(question_encoded, knowledge, 
                                                   self.batch_size)


        concatenated_output = torch.cat((vision_output, knowledge_encoded*0), 1)
        concatenated_output = torch.permute(concatenated_output, (0, 2, 1))
        #print(concatenated_output.size())
        output = self.classifier1(concatenated_output).squeeze(-1)
        prediction = self.classifier2(output + classification_output)
        
        return prediction

UniterM = UniterModel(config, img_d)
model_Uniter = Uniter_classifier(UniterM, len(list(answers_ids.keys()))).to(device)

In [ ]:
#no knowledge
import torch.nn.functional as F

#384 worked well (80% validation, 75.4%)
#336 (81.5% validation, 79.26%)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())
        self.wk = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())
        self.wv = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())

        self.dense = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())

    def positional_encoding(self, position, d_model):
        def get_angles(pos, i, d_model):
            angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
            return pos * angle_rates
        angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                                np.arange(d_model)[np.newaxis, :],
                                d_model)

        # apply sin to even indices in the array; 2i
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

        # apply cos to odd indices in the array; 2i+1
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

        pos_encoding = angle_rads[np.newaxis, ...]
        return torch.Tensor(pos_encoding)
    
    def scaled_dot_product_attention(self, q, k, v):
        matmul_qk = torch.matmul(q, torch.transpose(k, 2, 3))
        dk = matmul_qk.shape[-1]
        dk = torch.sqrt(torch.tensor(dk).to(device))+1e-6
        scaled_attention_logits = matmul_qk / dk
        attention_weights = F.softmax(scaled_attention_logits, dim=-1)
        output = torch.matmul(attention_weights, v)  

        return output, attention_weights 

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = torch.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return x.permute([0, 2, 1, 3])

    def forward(self, v, k, q):  # Note: original code call(self, v, k, q, mask):
        batch_size = q.shape[0]
        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)
        
        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
        
        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = self.scaled_dot_product_attention(
            q, k, v)

        scaled_attention = torch.permute(scaled_attention, (0, 2, 1, 3))  # (batch_size, seq_len_q, num_heads, depth)
        concat_attention = torch.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)
        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights


class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, dff, batch_size, 
                 maximum_position_encoding, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.layers_list = [nn.Linear(d_model, dff),nn.ReLU(),nn.Linear(dff, d_model)]
        self.ffn = nn.Sequential(*self.layers_list)
        self.maximum_position_encoding = int(maximum_position_encoding)

        self.layernorm1 = nn.LayerNorm(normalized_shape=(
                                                         maximum_position_encoding,d_model), eps=1e-6)
        self.layernorm2 = nn.LayerNorm(normalized_shape=(
                                                         maximum_position_encoding,d_model), eps=1e-6)

        self.dropout1 = nn.Dropout(rate)
        self.dropout2 = nn.Dropout(rate)

    def forward(self, question, img):
        attn_output, attn_weights_block = self.mha(img, question, question) 
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(question + attn_output)  

        
        ffn_output = self.ffn(out1)  
        ffn_output = self.dropout2(ffn_output)
        out2 = self.layernorm2(out1 + ffn_output)  

        return out2, attn_weights_block



class Encoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff,
                 maximum_position_encoding, batch_size, rate=0.1):
        super().__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.batch_size = batch_size

        self.question_dense = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())
        
        #self.pos_encoding = torch.Tensor(positional_encoding(maximum_position_encoding,
        #                                        self.d_model)).to(device)

        self.enc_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, 
                                                      dff, self.batch_size, maximum_position_encoding,
                                                      rate)
                           for _ in range(int(num_layers))])
        self.dropout = nn.Dropout(rate)
    
    def forward(self, question, img):
        seq_len = question.shape[1]
        x = self.question_dense(question)
        #x *= torch.sqrt(torch.tensor(self.d_model).to(device))
        x = torch.mul(x, torch.sqrt(torch.tensor(self.d_model).to(device)))
        #x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x)
        

        attention_weights = {}

        for i in range(self.num_layers):
            x, block = self.enc_layers[i](x, img)
            attention_weights['encoder_layer{}_block'.format(i + 1)] = block
        return x, attention_weights  # (batch_size, input_seq_len, d_model)


class Knowledge_Question_Encoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff, pe_input, rate, batch_size,
                 multiplier=10):
        super().__init__()
        self.batch_size = batch_size
        self.maximum_position_encoding = pe_input
        self.multiplier = multiplier
        self.bilstm_dim = int(d_model/2)
        self.bilstm1 = nn.LSTM(1024, self.bilstm_dim, batch_first=True, bidirectional=True)
        self.bilstm2 = nn.LSTM(1024, self.bilstm_dim, batch_first=True, bidirectional=True)

        self.fc1 = nn.Linear(d_model, self.maximum_position_encoding * multiplier)
        self.kn_dense = nn.Sequential(nn.Linear(192 * multiplier, d_model), nn.ReLU())

        self.question_encoder = Encoder(
            num_layers=num_layers,
            d_model=d_model,
            num_heads=num_heads,
            dff=dff,
            maximum_position_encoding=self.maximum_position_encoding,
            batch_size=self.batch_size)
        
    def forward(self, question, kn, batch_size):
        question, _ = self.bilstm1(question) #(batch, 38, 336)
        kn, _ = self.bilstm2(kn) #(batch, 192, 336)
        kn = self.fc1(kn)
        kn = torch.reshape(kn, (kn.shape[0], self.maximum_position_encoding, 
                                192 * self.multiplier))
        kn = self.kn_dense(kn)

        context, _ = self.question_encoder(question, kn)
        return context, question



class Uniter_classifier(nn.Module):
    def __init__(self, model, ans_size):
        super(Uniter_classifier, self).__init__()
        self.batch_size = BATCH_SIZE
        self.knowledge_encoder = Knowledge_Question_Encoder(1, HIDDEN_SIZE, 12, 
                                                            INTERMEDIATE_SIZE, 
                                                            38, 0.1, self.batch_size)
        self.pre_trained = model

        self.classifier1 = nn.Sequential(nn.Linear(87, 1), nn.ReLU())
        self.classifier2 = nn.Linear(HIDDEN_SIZE, ans_size)


    def forward(self, inputs_ids, question_pos, visual_feats, visual_pos, question_encoded, 
                knowledge):
 
        classification_output, vision_output, _, _ = self.pre_trained(inputs_ids, question_pos, visual_feats, visual_pos)

        #classification_output = classification_output.unsqueeze(1)
        #print(classification_output.size())
        #print(vision_output.size())
        #print(knowledge.size())
        knowledge_encoded, question = self.knowledge_encoder(question_encoded, knowledge, 
                                                   self.batch_size)
        #print(question.size())
        #print(vision_output.size())
        concatenated_output = torch.cat((vision_output, question), 1)
        concatenated_output = torch.permute(concatenated_output, (0, 2, 1))
        output = self.classifier1(concatenated_output).squeeze(-1)
        prediction = self.classifier2(output + classification_output)
        
        return prediction

UniterM = UniterModel(config, img_d)
model_Uniter = Uniter_classifier(UniterM, len(list(answers_ids.keys()))).to(device)

In [ ]:
#whole_output
import torch.nn.functional as F

#384 worked well (80% validation, 75.4%)
#336 (81.5% validation, 79.26%)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())
        self.wk = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())
        self.wv = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())

        self.dense = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())

    def positional_encoding(self, position, d_model):
        def get_angles(pos, i, d_model):
            angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
            return pos * angle_rates
        angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                                np.arange(d_model)[np.newaxis, :],
                                d_model)

        # apply sin to even indices in the array; 2i
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

        # apply cos to odd indices in the array; 2i+1
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

        pos_encoding = angle_rads[np.newaxis, ...]
        return torch.Tensor(pos_encoding)
    
    def scaled_dot_product_attention(self, q, k, v):
        matmul_qk = torch.matmul(q, torch.transpose(k, 2, 3))
        dk = matmul_qk.shape[-1]
        dk = torch.sqrt(torch.tensor(dk).to(device))+1e-6
        scaled_attention_logits = matmul_qk / dk
        attention_weights = F.softmax(scaled_attention_logits, dim=-1)
        output = torch.matmul(attention_weights, v)  

        return output, attention_weights 

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = torch.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return x.permute([0, 2, 1, 3])

    def forward(self, v, k, q):  # Note: original code call(self, v, k, q, mask):
        batch_size = q.shape[0]
        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)
        
        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
        
        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = self.scaled_dot_product_attention(
            q, k, v)

        scaled_attention = torch.permute(scaled_attention, (0, 2, 1, 3))  # (batch_size, seq_len_q, num_heads, depth)
        concat_attention = torch.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)
        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights


class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, dff, batch_size, 
                 maximum_position_encoding, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.layers_list = [nn.Linear(d_model, dff),nn.ReLU(),nn.Linear(dff, d_model)]
        self.ffn = nn.Sequential(*self.layers_list)
        self.maximum_position_encoding = int(maximum_position_encoding)

        self.layernorm1 = nn.LayerNorm(normalized_shape=(
                                                         maximum_position_encoding,d_model), eps=1e-6)
        self.layernorm2 = nn.LayerNorm(normalized_shape=(
                                                         maximum_position_encoding,d_model), eps=1e-6)

        self.dropout1 = nn.Dropout(rate)
        self.dropout2 = nn.Dropout(rate)

    def forward(self, question, img):
        attn_output, attn_weights_block = self.mha(img, question, question) 
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(question + attn_output)  

        
        ffn_output = self.ffn(out1)  
        ffn_output = self.dropout2(ffn_output)
        out2 = self.layernorm2(out1 + ffn_output)  

        return out2, attn_weights_block



class Encoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff,
                 maximum_position_encoding, batch_size, rate=0.1):
        super().__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.batch_size = batch_size

        self.question_dense = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())
        
        #self.pos_encoding = torch.Tensor(positional_encoding(maximum_position_encoding,
        #                                        self.d_model)).to(device)

        self.enc_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, 
                                                      dff, self.batch_size, maximum_position_encoding,
                                                      rate)
                           for _ in range(int(num_layers))])
        self.dropout = nn.Dropout(rate)
    
    def forward(self, question, img):
        seq_len = question.shape[1]
        x = self.question_dense(question)
        #x *= torch.sqrt(torch.tensor(self.d_model).to(device))
        x = torch.mul(x, torch.sqrt(torch.tensor(self.d_model).to(device)))
        #x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x)
        

        attention_weights = {}

        for i in range(self.num_layers):
            x, block = self.enc_layers[i](x, img)
            attention_weights['encoder_layer{}_block'.format(i + 1)] = block
        return x, attention_weights  # (batch_size, input_seq_len, d_model)


class Knowledge_Question_Encoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff, pe_input, rate, batch_size,
                 multiplier=10):
        super().__init__()
        self.batch_size = batch_size
        self.maximum_position_encoding = pe_input
        self.multiplier = multiplier
        self.bilstm_dim = int(d_model/2)
        self.bilstm1 = nn.LSTM(1024, self.bilstm_dim, batch_first=True, bidirectional=True)
        self.bilstm2 = nn.LSTM(1024, self.bilstm_dim, batch_first=True, bidirectional=True)

        self.fc1 = nn.Linear(d_model, self.maximum_position_encoding * multiplier)
        self.kn_dense = nn.Sequential(nn.Linear(192 * multiplier, d_model), nn.ReLU())

        self.question_encoder = Encoder(
            num_layers=num_layers,
            d_model=d_model,
            num_heads=num_heads,
            dff=dff,
            maximum_position_encoding=self.maximum_position_encoding,
            batch_size=self.batch_size)
        
    def forward(self, question, kn, batch_size):
        question, _ = self.bilstm1(question) #(batch, 38, 336)
        kn, _ = self.bilstm2(kn) #(batch, 192, 336)
        kn = self.fc1(kn)
        kn = torch.reshape(kn, (kn.shape[0], self.maximum_position_encoding, 
                                192 * self.multiplier))
        kn = self.kn_dense(kn)

        context, _ = self.question_encoder(question, kn)
        return context



class Uniter_classifier(nn.Module):
    def __init__(self, model, ans_size):
        super(Uniter_classifier, self).__init__()
        self.batch_size = BATCH_SIZE
        self.knowledge_encoder = Knowledge_Question_Encoder(1, HIDDEN_SIZE, 12, 
                                                            INTERMEDIATE_SIZE, 
                                                            38, 0.1, self.batch_size)
        self.pre_trained = model

        self.classifier1 = nn.Sequential(nn.Linear(38, 1), nn.ReLU())
        self.pool_classifier = nn.Sequential(nn.Linear(70, 1), nn.ReLU())
        self.classifier2 = nn.Linear(HIDDEN_SIZE, ans_size)


    def forward(self, inputs_ids, question_pos, visual_feats, visual_pos, question_encoded, 
                knowledge):
 
        classification_output, vision_output, _, pooled_output = self.pre_trained(inputs_ids, question_pos, visual_feats, visual_pos)

        #classification_output = classification_output.unsqueeze(1)
        #print(classification_output.size())
        #print(vision_output.size())
        #print(knowledge.size())
        knowledge_encoded = self.knowledge_encoder(question_encoded, knowledge, 
                                                   self.batch_size)


        #concatenated_output = torch.cat((vision_output, knowledge_encoded), 1)
        concatenated_output = torch.permute(knowledge_encoded, (0, 2, 1))

        #print(concatenated_output.size())
        output = self.classifier1(concatenated_output).squeeze(-1)
        #print(concatenated_output.size())
        #print(output.size())
        #print(pooled_output.size())
        pooled_output = self.pool_classifier(torch.permute(pooled_output, (0, 2, 1))).squeeze(-1)
        #print(pooled_output.size())
        prediction = self.classifier2(pooled_output+output)
        
        return prediction

UniterM = UniterModel(config, img_d)
model_Uniter = Uniter_classifier(UniterM, len(list(answers_ids.keys()))).to(device)

In [ ]:
#pool_output
import torch.nn.functional as F

#384 worked well (80% validation, 75.4%)
#336 (81.5% validation, 79.26%)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())
        self.wk = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())
        self.wv = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())

        self.dense = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())

    def positional_encoding(self, position, d_model):
        def get_angles(pos, i, d_model):
            angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
            return pos * angle_rates
        angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                                np.arange(d_model)[np.newaxis, :],
                                d_model)

        # apply sin to even indices in the array; 2i
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

        # apply cos to odd indices in the array; 2i+1
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

        pos_encoding = angle_rads[np.newaxis, ...]
        return torch.Tensor(pos_encoding)
    
    def scaled_dot_product_attention(self, q, k, v):
        matmul_qk = torch.matmul(q, torch.transpose(k, 2, 3))
        dk = matmul_qk.shape[-1]
        dk = torch.sqrt(torch.tensor(dk).to(device))+1e-6
        scaled_attention_logits = matmul_qk / dk
        attention_weights = F.softmax(scaled_attention_logits, dim=-1)
        output = torch.matmul(attention_weights, v)  

        return output, attention_weights 

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = torch.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return x.permute([0, 2, 1, 3])

    def forward(self, v, k, q):  # Note: original code call(self, v, k, q, mask):
        batch_size = q.shape[0]
        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)
        
        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
        
        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = self.scaled_dot_product_attention(
            q, k, v)

        scaled_attention = torch.permute(scaled_attention, (0, 2, 1, 3))  # (batch_size, seq_len_q, num_heads, depth)
        concat_attention = torch.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)
        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights


class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, dff, batch_size, 
                 maximum_position_encoding, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.layers_list = [nn.Linear(d_model, dff),nn.ReLU(),nn.Linear(dff, d_model)]
        self.ffn = nn.Sequential(*self.layers_list)
        self.maximum_position_encoding = int(maximum_position_encoding)

        self.layernorm1 = nn.LayerNorm(normalized_shape=(
                                                         maximum_position_encoding,d_model), eps=1e-6)
        self.layernorm2 = nn.LayerNorm(normalized_shape=(
                                                         maximum_position_encoding,d_model), eps=1e-6)

        self.dropout1 = nn.Dropout(rate)
        self.dropout2 = nn.Dropout(rate)

    def forward(self, question, img):
        attn_output, attn_weights_block = self.mha(img, question, question) 
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(question + attn_output)  

        
        ffn_output = self.ffn(out1)  
        ffn_output = self.dropout2(ffn_output)
        out2 = self.layernorm2(out1 + ffn_output)  

        return out2, attn_weights_block



class Encoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff,
                 maximum_position_encoding, batch_size, rate=0.1):
        super().__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.batch_size = batch_size

        self.question_dense = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())
        
        #self.pos_encoding = torch.Tensor(positional_encoding(maximum_position_encoding,
        #                                        self.d_model)).to(device)

        self.enc_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, 
                                                      dff, self.batch_size, maximum_position_encoding,
                                                      rate)
                           for _ in range(int(num_layers))])
        self.dropout = nn.Dropout(rate)
    
    def forward(self, question, img):
        seq_len = question.shape[1]
        x = self.question_dense(question)
        #x *= torch.sqrt(torch.tensor(self.d_model).to(device))
        x = torch.mul(x, torch.sqrt(torch.tensor(self.d_model).to(device)))
        #x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x)
        

        attention_weights = {}

        for i in range(self.num_layers):
            x, block = self.enc_layers[i](x, img)
            attention_weights['encoder_layer{}_block'.format(i + 1)] = block
        return x, attention_weights  # (batch_size, input_seq_len, d_model)


class Knowledge_Question_Encoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff, pe_input, rate, batch_size,
                 multiplier=10):
        super().__init__()
        self.batch_size = batch_size
        self.maximum_position_encoding = pe_input
        self.multiplier = multiplier
        self.bilstm_dim = int(d_model/2)
        self.bilstm1 = nn.LSTM(1024, self.bilstm_dim, batch_first=True, bidirectional=True)
        self.bilstm2 = nn.LSTM(1024, self.bilstm_dim, batch_first=True, bidirectional=True)

        self.fc1 = nn.Linear(d_model, self.maximum_position_encoding * multiplier)
        self.kn_dense = nn.Sequential(nn.Linear(192 * multiplier, d_model), nn.ReLU())

        self.question_encoder = Encoder(
            num_layers=num_layers,
            d_model=d_model,
            num_heads=num_heads,
            dff=dff,
            maximum_position_encoding=self.maximum_position_encoding,
            batch_size=self.batch_size)
        
    def forward(self, question, kn, batch_size):
        question, _ = self.bilstm1(question) #(batch, 38, 336)
        kn, _ = self.bilstm2(kn) #(batch, 192, 336)
        kn = self.fc1(kn)
        kn = torch.reshape(kn, (kn.shape[0], self.maximum_position_encoding, 
                                192 * self.multiplier))
        kn = self.kn_dense(kn)

        context, _ = self.question_encoder(question, kn)
        return context



class Uniter_classifier(nn.Module):
    def __init__(self, model, ans_size):
        super(Uniter_classifier, self).__init__()
        self.batch_size = BATCH_SIZE
        self.knowledge_encoder = Knowledge_Question_Encoder(1, HIDDEN_SIZE, 12, 
                                                            INTERMEDIATE_SIZE, 
                                                            38, 0.1, self.batch_size)
        self.pre_trained = model

        self.classifier1 = nn.Sequential(nn.Linear(38, 1), nn.ReLU())
        self.classifier2 = nn.Linear(HIDDEN_SIZE, ans_size)


    def forward(self, inputs_ids, question_pos, visual_feats, visual_pos, question_encoded, 
                knowledge):
 
        classification_output, vision_output, _, pooled_output = self.pre_trained(inputs_ids, question_pos, visual_feats, visual_pos)

        #classification_output = classification_output.unsqueeze(1)
        #print(classification_output.size())
        #print(vision_output.size())
        #print(knowledge.size())
        knowledge_encoded = self.knowledge_encoder(question_encoded, knowledge, 
                                                   self.batch_size)


        #concatenated_output = torch.cat((vision_output, knowledge_encoded), 1)
        concatenated_output = torch.permute(knowledge_encoded, (0, 2, 1))

        #print(concatenated_output.size())
        output = self.classifier1(concatenated_output).squeeze(-1)
        #print(concatenated_output.size())
        #print(output.size())
        #print(classification_output.size())
        prediction = self.classifier2(classification_output + output)
        
        return prediction

UniterM = UniterModel(config, img_d)
model_Uniter = Uniter_classifier(UniterM, len(list(answers_ids.keys()))).to(device)

In [ ]:
#no cross modality residual
import torch.nn.functional as F

#384 worked well (80% validation, 75.4%)
#336 (81.5% validation, 79.26%)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())
        self.wk = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())
        self.wv = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())

        self.dense = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())

    def positional_encoding(self, position, d_model):
        def get_angles(pos, i, d_model):
            angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
            return pos * angle_rates
        angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                                np.arange(d_model)[np.newaxis, :],
                                d_model)

        # apply sin to even indices in the array; 2i
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

        # apply cos to odd indices in the array; 2i+1
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

        pos_encoding = angle_rads[np.newaxis, ...]
        return torch.Tensor(pos_encoding)
    
    def scaled_dot_product_attention(self, q, k, v):
        matmul_qk = torch.matmul(q, torch.transpose(k, 2, 3))
        dk = matmul_qk.shape[-1]
        dk = torch.sqrt(torch.tensor(dk).to(device))+1e-6
        scaled_attention_logits = matmul_qk / dk
        attention_weights = F.softmax(scaled_attention_logits, dim=-1)
        output = torch.matmul(attention_weights, v)  

        return output, attention_weights 

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = torch.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return x.permute([0, 2, 1, 3])

    def forward(self, v, k, q):  # Note: original code call(self, v, k, q, mask):
        batch_size = q.shape[0]
        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)
        
        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
        
        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = self.scaled_dot_product_attention(
            q, k, v)

        scaled_attention = torch.permute(scaled_attention, (0, 2, 1, 3))  # (batch_size, seq_len_q, num_heads, depth)
        concat_attention = torch.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)
        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights


class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, dff, batch_size, 
                 maximum_position_encoding, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.layers_list = [nn.Linear(d_model, dff),nn.ReLU(),nn.Linear(dff, d_model)]
        self.ffn = nn.Sequential(*self.layers_list)
        self.maximum_position_encoding = int(maximum_position_encoding)

        self.layernorm1 = nn.LayerNorm(normalized_shape=(
                                                         maximum_position_encoding,d_model), eps=1e-6)
        self.layernorm2 = nn.LayerNorm(normalized_shape=(
                                                         maximum_position_encoding,d_model), eps=1e-6)

        self.dropout1 = nn.Dropout(rate)
        self.dropout2 = nn.Dropout(rate)

    def forward(self, question, img):
        attn_output, attn_weights_block = self.mha(img, question, question) 
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(question + attn_output)  

        
        ffn_output = self.ffn(out1)  
        ffn_output = self.dropout2(ffn_output)
        out2 = self.layernorm2(out1 + ffn_output)  

        return out2, attn_weights_block



class Encoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff,
                 maximum_position_encoding, batch_size, rate=0.1):
        super().__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.batch_size = batch_size

        self.question_dense = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())
        
        #self.pos_encoding = torch.Tensor(positional_encoding(maximum_position_encoding,
        #                                        self.d_model)).to(device)

        self.enc_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, 
                                                      dff, self.batch_size, maximum_position_encoding,
                                                      rate)
                           for _ in range(int(num_layers))])
        self.dropout = nn.Dropout(rate)
    
    def forward(self, question, img):
        seq_len = question.shape[1]
        x = self.question_dense(question)
        #x *= torch.sqrt(torch.tensor(self.d_model).to(device))
        x = torch.mul(x, torch.sqrt(torch.tensor(self.d_model).to(device)))

        #x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x)
        

        attention_weights = {}

        for i in range(self.num_layers):
            x, block = self.enc_layers[i](x, img)
            attention_weights['encoder_layer{}_block'.format(i + 1)] = block
        return x, attention_weights  # (batch_size, input_seq_len, d_model)


class Knowledge_Question_Encoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff, pe_input, rate, batch_size,
                 multiplier=10):
        super().__init__()
        self.batch_size = batch_size
        self.maximum_position_encoding = pe_input
        self.multiplier = multiplier
        self.bilstm_dim = int(d_model/2)
        self.bilstm1 = nn.LSTM(1024, self.bilstm_dim, batch_first=True, bidirectional=True)
        self.bilstm2 = nn.LSTM(1024, self.bilstm_dim, batch_first=True, bidirectional=True)

        self.fc1 = nn.Linear(d_model, self.maximum_position_encoding * multiplier)
        self.kn_dense = nn.Sequential(nn.Linear(192 * multiplier, d_model), nn.ReLU())

        self.question_encoder = Encoder(
            num_layers=num_layers,
            d_model=d_model,
            num_heads=num_heads,
            dff=dff,
            maximum_position_encoding=self.maximum_position_encoding,
            batch_size=self.batch_size)
        
    def forward(self, question, kn, batch_size):
        question, _ = self.bilstm1(question) #(batch, 38, 336)
        kn, _ = self.bilstm2(kn) #(batch, 192, 336)
        kn = self.fc1(kn)
        kn = torch.reshape(kn, (kn.shape[0], self.maximum_position_encoding, 
                                192 * self.multiplier))
        kn = self.kn_dense(kn)

        context, _ = self.question_encoder(question, kn)
        return context



class Uniter_classifier(nn.Module):
    def __init__(self, model, ans_size):
        super(Uniter_classifier, self).__init__()
        self.batch_size = BATCH_SIZE
        self.knowledge_encoder = Knowledge_Question_Encoder(1, HIDDEN_SIZE, 12, 
                                                            INTERMEDIATE_SIZE, 
                                                            38, 0.1, self.batch_size)
        self.pre_trained = model

        self.classifier1 = nn.Sequential(nn.Linear(87, 1), nn.ReLU())
        self.classifier2 = nn.Linear(HIDDEN_SIZE, ans_size)


    def forward(self, inputs_ids, question_pos, visual_feats, visual_pos, question_encoded, 
                knowledge):
 
        classification_output, vision_output, _, _ = self.pre_trained(inputs_ids, question_pos, visual_feats, visual_pos)

        #classification_output = classification_output.unsqueeze(1)
        #print(classification_output.size())
        #print(vision_output.size())
        #print(knowledge.size())
        knowledge_encoded = self.knowledge_encoder(question_encoded, knowledge, 
                                                   self.batch_size)


        concatenated_output = torch.cat((vision_output, knowledge_encoded), 1)
        concatenated_output = torch.permute(concatenated_output, (0, 2, 1))
        output = self.classifier1(concatenated_output).squeeze(-1)
        prediction = self.classifier2(output)
        
        return prediction

UniterM = UniterModel(config, img_d)
model_Uniter = Uniter_classifier(UniterM, len(list(answers_ids.keys()))).to(device)

In [ ]:

import torch.nn.functional as F




device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())
        self.wk = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())
        self.wv = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())

        self.dense = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())

    def positional_encoding(self, position, d_model):
        def get_angles(pos, i, d_model):
            angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
            return pos * angle_rates
        angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                                np.arange(d_model)[np.newaxis, :],
                                d_model)

        # apply sin to even indices in the array; 2i
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

        # apply cos to odd indices in the array; 2i+1
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

        pos_encoding = angle_rads[np.newaxis, ...]
        return torch.Tensor(pos_encoding)
    
    def scaled_dot_product_attention(self, q, k, v):
        matmul_qk = torch.matmul(q, torch.transpose(k, 2, 3))
        dk = matmul_qk.shape[-1]
        dk = torch.sqrt(torch.tensor(dk).to(device))+1e-6
        scaled_attention_logits = matmul_qk / dk
        attention_weights = F.softmax(scaled_attention_logits, dim=-1)
        output = torch.matmul(attention_weights, v)  

        return output, attention_weights 

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = torch.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return x.permute([0, 2, 1, 3])

    def forward(self, v, k, q):  # Note: original code call(self, v, k, q, mask):
        batch_size = q.shape[0]
        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)
        
        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
        
        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = self.scaled_dot_product_attention(
            q, k, v)

        scaled_attention = torch.permute(scaled_attention, (0, 2, 1, 3))  # (batch_size, seq_len_q, num_heads, depth)
        concat_attention = torch.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)
        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights


class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, dff, batch_size, 
                 maximum_position_encoding, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.layers_list = [nn.Linear(d_model, dff),nn.ReLU(),nn.Linear(dff, d_model)]
        self.ffn = nn.Sequential(*self.layers_list)
        self.maximum_position_encoding = int(maximum_position_encoding)

        self.layernorm1 = nn.LayerNorm(normalized_shape=(
                                                         maximum_position_encoding,d_model), eps=1e-6)
        self.layernorm2 = nn.LayerNorm(normalized_shape=(
                                                         maximum_position_encoding,d_model), eps=1e-6)

        self.dropout1 = nn.Dropout(rate)
        self.dropout2 = nn.Dropout(rate)

    def forward(self, question, img):
        attn_output, attn_weights_block = self.mha(img, question, question) 
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(question + attn_output)  

        
        ffn_output = self.ffn(out1)  
        ffn_output = self.dropout2(ffn_output)
        out2 = self.layernorm2(out1 + ffn_output)  

        return out2, attn_weights_block



class Encoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff,
                 maximum_position_encoding, batch_size, rate=0.1):
        super().__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.batch_size = batch_size

        self.question_dense = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU())
        
        #self.pos_encoding = torch.Tensor(positional_encoding(maximum_position_encoding,
        #                                        self.d_model)).to(device)

        self.enc_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, 
                                                      dff, self.batch_size, maximum_position_encoding,
                                                      rate)
                           for _ in range(int(num_layers))])
        self.dropout = nn.Dropout(rate)
    
    def forward(self, question, img):
        seq_len = question.shape[1]
        x = self.question_dense(question)
        #x *= torch.sqrt(torch.tensor(self.d_model).to(device))
        x = torch.mul(x, torch.sqrt(torch.tensor(self.d_model).to(device)))

        #x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x)
        

        attention_weights = {}

        for i in range(self.num_layers):
            x, block = self.enc_layers[i](x, img)
            attention_weights['encoder_layer{}_block'.format(i + 1)] = block
        return x, attention_weights  # (batch_size, input_seq_len, d_model)


class Knowledge_Question_Encoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, dff, pe_input, rate, batch_size,
                 multiplier=10):
        super().__init__()
        self.batch_size = batch_size
        self.maximum_position_encoding = pe_input
        self.multiplier = multiplier
        self.bilstm_dim = int(d_model/2)
        self.bilstm1 = nn.LSTM(1024, self.bilstm_dim, batch_first=True, bidirectional=True)
        self.bilstm2 = nn.LSTM(1024, self.bilstm_dim, batch_first=True, bidirectional=True)

        self.fc1 = nn.Linear(d_model, self.maximum_position_encoding * multiplier)
        self.kn_dense = nn.Sequential(nn.Linear(192 * multiplier, d_model), nn.ReLU())

        self.question_encoder = Encoder(
            num_layers=num_layers,
            d_model=d_model,
            num_heads=num_heads,
            dff=dff,
            maximum_position_encoding=self.maximum_position_encoding,
            batch_size=self.batch_size)
        
    def forward(self, question, kn, batch_size):
        question, _ = self.bilstm1(question) #(batch, 38, 336)
        kn, _ = self.bilstm2(kn) #(batch, 192, 336)
        kn = self.fc1(kn)
        kn = torch.reshape(kn, (kn.shape[0], self.maximum_position_encoding, 
                                192 * self.multiplier))
        kn = self.kn_dense(kn)

        context, _ = self.question_encoder(question, kn)
        return context



class Uniter_classifier(nn.Module):
    def __init__(self, model, ans_size):
        super(Uniter_classifier, self).__init__()
        self.batch_size = BATCH_SIZE
        self.knowledge_encoder = Knowledge_Question_Encoder(1, HIDDEN_SIZE, 12, 
                                                            INTERMEDIATE_SIZE, 
                                                            38, 0.1, self.batch_size)
        self.pre_trained = model

        self.classifier1 = nn.Sequential(nn.Linear(38+img_proposal, 1), nn.ReLU())
        self.classifier2 = nn.Linear(HIDDEN_SIZE, ans_size)


    def forward(self, inputs_ids, question_pos, visual_feats, visual_pos, question_encoded, 
                knowledge):
 
        classification_output, vision_output, _, _ = self.pre_trained(inputs_ids, question_pos, visual_feats, visual_pos)

        #classification_output = classification_output.unsqueeze(1)
        #print(classification_output.size())
        #print(vision_output.size())
        #print(knowledge.size())
        knowledge_encoded = self.knowledge_encoder(question_encoded, knowledge, 
                                                   self.batch_size)


        concatenated_output = torch.cat((vision_output, knowledge_encoded), 1)
        concatenated_output = torch.permute(concatenated_output, (0, 2, 1))
        #print(concatenated_output.size())
        output = self.classifier1(concatenated_output).squeeze(-1)
        prediction = self.classifier2(output + classification_output)
        
        return prediction

UniterM = UniterModel(config, img_d)
model_Uniter = Uniter_classifier(UniterM, len(list(answers_ids.keys()))).to(device)

In [ ]:
torch.cuda.empty_cache()

idx, batch = next(enumerate(train_dataloader))
question = torch.stack(batch['question']).transpose(0,1).to(device)
question_pos = batch['question_pos'].to(device)
image = batch['image'].to(device)
position = batch['position'].to(device)
question_encoded = batch['question_encoded'].to(device)
knowledge = batch['knowledge'].to(device)
model_Uniter(question, question_pos, image, position, question_encoded, knowledge)

tensor([[-0.2077,  0.0634,  0.2065,  0.6202],
        [-0.0553,  0.1553,  0.3844,  0.5094],
        [-0.2786, -0.1208,  0.2827,  0.4091],
        [-0.2881,  0.0431,  0.4489,  0.4419],
        [-0.3570,  0.1583,  0.2147,  0.5296],
        [-0.3085,  0.1114,  0.3555,  0.5602],
        [-0.3405,  0.1240,  0.2417,  0.5516],
        [-0.3640, -0.0539,  0.3020,  0.5204],
        [-0.1919,  0.0568,  0.3953,  0.4620],
        [-0.3197, -0.0929,  0.2122,  0.3986],
        [-0.0766, -0.0081,  0.0776,  0.5572],
        [-0.1466, -0.0462,  0.2666,  0.4456],
        [-0.4317, -0.0653,  0.2788,  0.5346],
        [-0.0892,  0.1892,  0.2840,  0.6132],
        [-0.1463,  0.2027,  0.2333,  0.5953],
        [-0.1755,  0.1877,  0.2971,  0.3998]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def evaluate(model, loader, record_f1=None):
    predictions = None
    true = None
    torch.cuda.empty_cache()
    model.eval()
    for idx, batch in tqdm(enumerate(loader)):

        question = torch.stack(batch['question']).transpose(0,1).to(device)
        question_pos = batch['question_pos'].to(device)
        image = batch['image'].to(device)
        position = batch['position'].to(device)
        question_encoded = batch['question_encoded'].to(device)
        knowledge = batch['knowledge'].to(device)
        answer = batch['answer'].to(device)
        prediction = model(question, question_pos, image, position, question_encoded, knowledge)


        if predictions is not None:
            prediction = prediction.detach().cpu().numpy()
            predictions = np.append(predictions, np.argmax(prediction, axis=1))
            true = np.append(true, answer.detach().cpu().numpy())
        else:
            predictions = np.argmax(prediction.detach().cpu().numpy(), axis=1)
            true = answer.detach().cpu().numpy()
    #print(true.shape())
    #print(len(predictions.shape))
    #print(predictions)
    f_1_score = f1_score(predictions, true, average='micro')
    print("Accuracy is:", accuracy_score(predictions, true))
    print("F1-metric is:", f_1_score)
    if record_f1:
        return f_1_score


!mkdir /content/best_model
def save_model(model):
    PATH = '/content/best_model/best_f1.pt'
    torch.save({
            'model_state_dict': model.state_dict()
            }, PATH)

mkdir: cannot create directory ‘/content/best_model’: File exists


In [ ]:
from tqdm import tqdm
from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler
# define the final training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model_Uniter.parameters(), lr=0.0001)

scaler = GradScaler()

best_f1 = 0.0
EPOCHS = 50
for epoch in range(EPOCHS):
    #epoch_loss = 0.0
    model_Uniter.train()
    print("epoch: ", epoch)
    for idx, batch in tqdm(enumerate(train_dataloader)):
        
        question = torch.stack(batch['question']).transpose(0,1).to(device)
        question_pos = batch['question_pos'].to(device)
        image = batch['image'].to(device)
        position = batch['position'].to(device)
        answer = batch['answer'].to(device)
        question_encoded = batch['question_encoded'].to(device)
        knowledge = batch['knowledge'].to(device)

        optimizer.zero_grad()
        with autocast():
            prediction = model_Uniter(question, question_pos, image, position, question_encoded, knowledge)
            loss = criterion(prediction, answer)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        #epoch_loss += loss.item()
    #print('Epoch loss is:', epoch_loss/len(df_train))
    f1_temp = evaluate(model_Uniter, val_dataloader, True)
    if f1_temp > best_f1:
        best_f1 = f1_temp
        save_model(model_Uniter)
        print('New best f1!')
    print('\n')

epoch:  0


531it [00:42, 12.36it/s]
307it [00:16, 19.07it/s]


Accuracy is: 0.5504998979800041
F1-metric is: 0.5504998979800041
New best f1!


epoch:  1


531it [00:42, 12.40it/s]
307it [00:16, 19.00it/s]


Accuracy is: 0.5504998979800041
F1-metric is: 0.5504998979800041


epoch:  2


...

Accuracy is: 0.4413385023464599
F1-metric is: 0.4413385023464599


epoch:  48


531it [00:42, 12.44it/s]
307it [00:16, 19.12it/s]


Accuracy is: 0.42052642317894307
F1-metric is: 0.42052642317894307


epoch:  49


531it [00:42, 12.49it/s]
307it [00:15, 19.19it/s]

Accuracy is: 0.41032442358702303
F1-metric is: 0.41032442358702303




In [ ]:
#Layer =2,h_size=192,pool_activation=gelu, vgg
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



369it [00:25, 14.66it/s]

Accuracy is: 0.6896200814111262
F1-metric is: 0.6896200814111262


In [ ]:
#yes/no, Layer =2,h_size=192,pool_activation=gelu, vgg
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



184it [00:09, 20.26it/s]

Accuracy is: 0.9322898945219462
F1-metric is: 0.9322898945219462


In [ ]:
#open ended, Layer =2,h_size=192,pool_activation=gelu, vgg
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



185it [00:09, 20.45it/s]

Accuracy is: 0.4264457220155563
F1-metric is: 0.4264457220155563


In [ ]:
#Layer =2,h_size=192,pool_activation=gelu, densenet
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



369it [00:21, 17.57it/s]

Accuracy is: 0.6972523744911805
F1-metric is: 0.6972523744911805


In [ ]:
#yes/no, Layer =2,h_size=192,pool_activation=gelu, densenet
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



184it [00:10, 18.19it/s]

Accuracy is: 0.9445389588295339
F1-metric is: 0.9445389588295339


In [ ]:
#open ended, Layer =2,h_size=192,pool_activation=gelu, densenet
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



185it [00:09, 19.06it/s]

Accuracy is: 0.43659113966858304
F1-metric is: 0.43659113966858304


In [ ]:
#Layer =2,h_size=192,pool_activation=gelu, inception
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



369it [00:20, 18.05it/s]

Accuracy is: 0.6955563093622795
F1-metric is: 0.6955563093622795


In [ ]:
#yes/no, Layer =2,h_size=192,pool_activation=gelu, inception
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



184it [00:11, 16.19it/s]

Accuracy is: 0.9316093909493025
F1-metric is: 0.9316093909493025


In [ ]:
#open ended, Layer =2,h_size=192,pool_activation=gelu, inception
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



185it [00:10, 18.36it/s]

Accuracy is: 0.4247548190733852
F1-metric is: 0.4247548190733852


In [ ]:
#Layer =2,h_size=192,pool_activation=gelu, no image
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



369it [00:27, 13.33it/s]


Accuracy is: 0.6307666214382632
F1-metric is: 0.6307666214382632


In [ ]:
#yes/no, Layer =2,h_size=192,pool_activation=gelu, no image
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



184it [00:16, 11.26it/s]

Accuracy is: 0.9336509016672337
F1-metric is: 0.9336509016672337


In [ ]:
#open ended, Layer =2,h_size=192,pool_activation=gelu, no image
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



185it [00:14, 12.61it/s]

Accuracy is: 0.31078796077105175
F1-metric is: 0.31078796077105175


In [ ]:
#Layer =2,h_size=192,pool_activation=gelu, no question
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



369it [00:19, 18.73it/s]

Accuracy is: 0.28256445047489825
F1-metric is: 0.28256445047489825


In [ ]:
#yes/no, Layer =2,h_size=192,pool_activation=gelu, no question
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



184it [00:09, 18.99it/s]

Accuracy is: 0.544743109901327
F1-metric is: 0.544743109901327


In [ ]:
#open ended, Layer =2,h_size=192,pool_activation=gelu, no question
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



185it [00:09, 19.35it/s]

Accuracy is: 0.17348664186675686
F1-metric is: 0.17348664186675686


In [ ]:
#Layer =2,h_size=192,pool_activation=gelu, only question
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



369it [00:20, 18.28it/s]

Accuracy is: 0.48795793758480327
F1-metric is: 0.48795793758480327


In [ ]:
#yes/no, Layer =2,h_size=192,pool_activation=gelu, only question
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



184it [00:09, 18.89it/s]

Accuracy is: 0.8383804014971079
F1-metric is: 0.8383804014971079


In [ ]:
#open ended, Layer =2,h_size=192,pool_activation=gelu, only question
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



185it [00:09, 18.90it/s]

Accuracy is: 0.13425769360838688
F1-metric is: 0.13425769360838688


In [ ]:
#Layer =2,h_size=192,pool_activation=gelu, rectified knowledge
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



369it [00:36, 10.10it/s]

Accuracy is: 0.6936906377204884
F1-metric is: 0.6936906377204884


In [ ]:
#yes_no, Layer =2,h_size=192,pool_activation=gelu, rectified knowledge
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



184it [00:09, 19.04it/s]

Accuracy is: 0.9367131677441306
F1-metric is: 0.9367131677441306


In [ ]:
#open_ended, Layer =2,h_size=192,pool_activation=gelu, rectified knowledge
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



185it [00:09, 19.14it/s]

Accuracy is: 0.4267839026039905
F1-metric is: 0.4267839026039905


In [ ]:
#Layer =2,h_size=192,pool_activation=gelu, resnet50, no cross modality residual
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



369it [00:24, 15.14it/s]

Accuracy is: 0.6918249660786974
F1-metric is: 0.6918249660786974


In [ ]:
#yes_no, Layer =2,h_size=192,pool_activation=gelu, resnet50, no cross modality residual
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



184it [00:09, 19.28it/s]

Accuracy is: 0.9319496427356244
F1-metric is: 0.9319496427356244


In [ ]:
#open_ended, Layer =2,h_size=192,pool_activation=gelu, resnet50, no cross modality residual
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



185it [00:09, 19.28it/s]

Accuracy is: 0.43151843084206964
F1-metric is: 0.43151843084206964


In [ ]:
#pooled_output, no visual embedding, Layer =2,h_size=192,pool_activation=gelu
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



369it [00:19, 18.85it/s]


Accuracy is: 0.69640434192673
F1-metric is: 0.69640434192673


In [ ]:
#yes_no, pooled_output, no visual embedding, Layer =2,h_size=192,pool_activation=gelu
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



184it [00:09, 19.39it/s]

Accuracy is: 0.9407961891799932
F1-metric is: 0.9407961891799932


In [ ]:
#open ended, pooled_output, no visual embedding, Layer =2,h_size=192,pool_activation=gelu
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



185it [00:09, 18.94it/s]

Accuracy is: 0.44267839026039907
F1-metric is: 0.44267839026039907


In [ ]:
#whole_output, no visual embedding, Layer =2,h_size=192,pool_activation=gelu
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



369it [00:26, 13.93it/s]

Accuracy is: 0.6957259158751696
F1-metric is: 0.6957259158751696


In [ ]:
#open_ended, whole_output, no visual embedding, Layer =2,h_size=192,pool_activation=gelu
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



185it [00:09, 19.87it/s]

Accuracy is: 0.4430165708488333
F1-metric is: 0.4430165708488333


In [ ]:
#yes_no, whole_output, no visual embedding, Layer =2,h_size=192,pool_activation=gelu
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



184it [00:09, 19.31it/s]

Accuracy is: 0.9227628445049336
F1-metric is: 0.9227628445049336


In [ ]:
#Layer =2,h_size=192,pool_activation=gelu, resnet50, no knowledge
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



369it [00:31, 11.77it/s]

Accuracy is: 0.6862279511533242
F1-metric is: 0.6862279511533242


In [ ]:
##open ended,Layer =2,h_size=192,pool_activation=gelu, resnet50, no knowledge
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



185it [00:09, 19.81it/s]

Accuracy is: 0.4230639161312141
F1-metric is: 0.423063916131214


In [ ]:
##yes_no,Layer =2,h_size=192,pool_activation=gelu, resnet50, no knowledge
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



184it [00:09, 18.66it/s]

Accuracy is: 0.9367131677441306
F1-metric is: 0.9367131677441306


In [ ]:
#Layer =3,h_size=192,pool_activation=gelu
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



369it [00:24, 14.84it/s]

Accuracy is: 0.6901289009497965
F1-metric is: 0.6901289009497965


In [ ]:
#yes_no, Layer =3,h_size=192,pool_activation=gelu
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



184it [00:16, 10.85it/s]

Accuracy is: 0.9404559373936713
F1-metric is: 0.9404559373936713


In [ ]:
#open_ended, Layer =3,h_size=192,pool_activation=gelu
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



185it [00:09, 19.17it/s]

Accuracy is: 0.4261075414271221
F1-metric is: 0.42610754142712215


In [ ]:
#Layer =2,h_size=192,pool_activation=gelu
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



369it [00:23, 15.39it/s]

Accuracy is: 0.694538670284939
F1-metric is: 0.694538670284939


In [ ]:
#yes_no, Layer =2,h_size=192,pool_activation=gelu
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



184it [00:10, 17.90it/s]

Accuracy is: 0.9333106498809118
F1-metric is: 0.9333106498809118


In [ ]:
#open_ended, Layer =2,h_size=192,pool_activation=gelu
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



185it [00:09, 19.49it/s]


Accuracy is: 0.43016570848833274
F1-metric is: 0.43016570848833274


In [ ]:
#yes_no, Layer =1,h_size=192,pool_activation=gelu
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



184it [00:12, 15.23it/s]

Accuracy is: 0.9275263695134399
F1-metric is: 0.9275263695134399


In [ ]:
#open_ended, Layer =1,h_size=192,pool_activation=gelu
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



185it [00:11, 15.69it/s]

Accuracy is: 0.41968211024687185
F1-metric is: 0.4196821102468719


In [ ]:
#Layer =1,h_size=192,pool_activation=gelu
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



369it [00:31, 11.74it/s]

Accuracy is: 0.6879240162822252
F1-metric is: 0.6879240162822252


In [ ]:
#Layer =2,h_size=768,pool_activation=gelu
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



369it [00:29, 12.69it/s]

Accuracy is: 0.5985413839891451
F1-metric is: 0.5985413839891451


In [ ]:
#Layer =1,h_size=768,pool_activation=gelu
print("TEST THE MODEL\n")
checkpoint = torch.load('/content/best_model/best_f1.pt')
model_Uniter.load_state_dict(checkpoint['model_state_dict'])
evaluate(model_Uniter.to(device), test_dataloader)

TEST THE MODEL



369it [00:28, 12.85it/s]

Accuracy is: 0.6300881953867028
F1-metric is: 0.6300881953867028


In [ ]:
!pip install torchcontrib

  Created wheel for torchcontrib: filename=torchcontrib-0.0.2-py3-none-any.whl size=7532 sha256=a633921b79e29abf05dd62b0f7b407be972615788208246fc4f8dc2d05cb5978
  Stored in directory: /root/.cache/pip/wheels/91/58/d0/f03811c3e34e1f14031294b5f30d8693689972af874d1225b8
Successfully built torchcontrib


In [ ]:
!pip install tqdm 